In [2]:
!nvidia-smi

Mon Jun  8 02:46:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# ELMo预训练模型的使用

In [4]:
%cd ./drive/My\ Drive/PyTorch

/content/drive/My Drive/PyTorch


In [8]:
!pip install allennlp

  Using cached https://files.pythonhosted.org/packages/41/5b/e07dd3bf104237bce4b398558b104c8e500333d6f30eabe3fa9685356b7d/spacy-2.1.9-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/34/46/b1d0bb71d308e820ed30316c5f0a017cb5ef5f4324bcbc7da3cf9d3b075c/blis-0.2.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/18/a5/9ace20422e7bb1bdcad31832ea85c52a09900cd4a7ce711246bfb92206ba/thinc-7.0.8-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/20/93/f222fb957764a283203525ef20e62008675fd0a14ffff8cc1b1490147c63/preshed-2.0.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.9 which is incompatible.
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled p

In [9]:
!pwd

/content/drive/My Drive/PyTorch


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
from allennlp.modules.elmo import Elmo, batch_to_ids
from allennlp.commands.elmo import ElmoEmbedder
from allennlp.nn.util import remove_sentence_boundaries

In [0]:
# 预训练模型下载

In [0]:
options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

In [0]:
# 获得ELMo编码器的类

In [14]:
elmo_bilm = ElmoEmbedder(options_file=options_file, weight_file=weight_file).elmo_bilm
elmo_bilm.cuda()
sentences = [['Taday', 'is', 'sunny', '.'], ['Hello', '!']]

100%|██████████| 374434792/374434792 [00:20<00:00, 18017369.50B/s]


In [15]:
elmo_bilm.cuda()

_ElmoBiLm(
  (_token_embedder): _ElmoCharacterEncoder(
    (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
    (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
    (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
    (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
    (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
    (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
    (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
    (_highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_features=4096, bias=True)
      )
    )
    (_projection): Linear(in_features=2048, out_features=512, bias=True)
  )
  (_elmo_lstm): ElmoLstm(
    (forward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, 

In [0]:
# 获得所有单词的字符ID，维度为batch_size(2)*max_sentence_len(4)*word_len(50)

In [27]:
character_ids = batch_to_ids(sentences).cuda();character_ids

tensor([[[259,  85,  98, 101,  98, 122, 260, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261],
         [259, 106, 116, 260, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261],
         [259, 116, 118, 111, 111, 122, 260, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261],
         [259,  47, 260, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 26

In [28]:
character_ids.shape

torch.Size([2, 4, 50])

In [0]:
# 获得ELMo输出

In [23]:
bilm_output = elmo_bilm(character_ids);bilm_output

{'activations': [tensor([[[ -3.1190,   7.4196, -14.9239,  ...,   2.8717,   5.6098,   6.6934],
           [ -0.7891,  -0.5492,   0.1154,  ...,  -0.0757,   0.0520,  -0.3345],
           [  0.1915,   0.2300,  -0.2894,  ...,  -0.0645,   0.5810,   0.2177],
           [  0.0589,  -0.6212,   0.0233,  ...,   0.6718,  -0.7320,   0.6754],
           [ -0.8872,  -0.2004,  -1.0601,  ...,  -0.2655,   0.2115,   0.1977],
           [ -6.0587,  13.7965, -15.6421,  ...,  21.0167,  16.7111,   6.5752]],
  
          [[ -3.1190,   7.4196, -14.9239,  ...,   2.8717,   5.6098,   6.6934],
           [  0.1743,  -1.4072,  -0.1939,  ...,  -0.6675,  -0.2126,   0.4458],
           [ -1.3351,   0.6297,  -1.4650,  ...,   0.0295,   0.6432,   0.8339],
           [ -6.0587,  13.7965, -15.6421,  ...,  21.0167,  16.7111,   6.5752],
           [ -0.0000,  -0.0000,   0.0000,  ...,   0.0000,  -0.0000,  -0.0000],
           [ -0.0000,  -0.0000,   0.0000,  ...,   0.0000,  -0.0000,  -0.0000]]],
         device='cuda:0'),
  te

In [0]:
# ELMo编码

In [26]:
layer_activations = bilm_output['activations'];layer_activations

[tensor([[[ -3.1190,   7.4196, -14.9239,  ...,   2.8717,   5.6098,   6.6934],
          [ -0.7891,  -0.5492,   0.1154,  ...,  -0.0757,   0.0520,  -0.3345],
          [  0.1915,   0.2300,  -0.2894,  ...,  -0.0645,   0.5810,   0.2177],
          [  0.0589,  -0.6212,   0.0233,  ...,   0.6718,  -0.7320,   0.6754],
          [ -0.8872,  -0.2004,  -1.0601,  ...,  -0.2655,   0.2115,   0.1977],
          [ -6.0587,  13.7965, -15.6421,  ...,  21.0167,  16.7111,   6.5752]],
 
         [[ -3.1190,   7.4196, -14.9239,  ...,   2.8717,   5.6098,   6.6934],
          [  0.1743,  -1.4072,  -0.1939,  ...,  -0.6675,  -0.2126,   0.4458],
          [ -1.3351,   0.6297,  -1.4650,  ...,   0.0295,   0.6432,   0.8339],
          [ -6.0587,  13.7965, -15.6421,  ...,  21.0167,  16.7111,   6.5752],
          [ -0.0000,  -0.0000,   0.0000,  ...,   0.0000,  -0.0000,  -0.0000],
          [ -0.0000,  -0.0000,   0.0000,  ...,   0.0000,  -0.0000,  -0.0000]]],
        device='cuda:0'),
 tensor([[[ 0.0570,  0.0479, -0.0

In [0]:
# 每个位置是否有单词

In [31]:
mask_with_bos_eos = bilm_output['mask'];mask_with_bos_eos

tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0]], device='cuda:0')

In [0]:
# 去掉ELMo加上的句子开始和结束符

In [36]:
without_bos_eos = [remove_sentence_boundaries(layer, mask_with_bos_eos) for layer in layer_activations];without_bos_eos

[(tensor([[[-0.7891, -0.5492,  0.1154,  ..., -0.0757,  0.0520, -0.3345],
           [ 0.1915,  0.2300, -0.2894,  ..., -0.0645,  0.5810,  0.2177],
           [ 0.0589, -0.6212,  0.0233,  ...,  0.6718, -0.7320,  0.6754],
           [-0.8872, -0.2004, -1.0601,  ..., -0.2655,  0.2115,  0.1977]],
  
          [[ 0.1743, -1.4072, -0.1939,  ..., -0.6675, -0.2126,  0.4458],
           [-1.3351,  0.6297, -1.4650,  ...,  0.0295,  0.6432,  0.8339],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
         device='cuda:0'), tensor([[1, 1, 1, 1],
          [1, 1, 0, 0]], device='cuda:0')),
 (tensor([[[-0.2670, -0.0881, -0.1534,  ...,  0.0034,  0.2856, -0.0127],
           [-0.4867, -0.2524, -0.0537,  ...,  0.1755,  0.9264,  0.7234],
           [-0.0058, -0.5936, -0.2885,  ...,  0.1183,  0.7336,  0.4271],
           [-0.9750, -0.3586, -0.0106,  ..., -0.0282, -0.0177,  0.0600]],
  
          [[ 0.0792, 

In [0]:
# 获得三层ELMo编码，每层1024维，维度为3*batch_size(2)*max_sentence_len(4)*1024

In [38]:
all_layers = torch.cat([ele[0].unsqueeze(0) for ele in without_bos_eos], dim=0);all_layers

tensor([[[[-7.8914e-01, -5.4923e-01,  1.1545e-01,  ..., -7.5666e-02,
            5.1954e-02, -3.3451e-01],
          [ 1.9155e-01,  2.2999e-01, -2.8944e-01,  ..., -6.4465e-02,
            5.8102e-01,  2.1768e-01],
          [ 5.8930e-02, -6.2122e-01,  2.3317e-02,  ...,  6.7177e-01,
           -7.3198e-01,  6.7544e-01],
          [-8.8715e-01, -2.0040e-01, -1.0601e+00,  ..., -2.6555e-01,
            2.1146e-01,  1.9773e-01]],

         [[ 1.7428e-01, -1.4072e+00, -1.9392e-01,  ..., -6.6748e-01,
           -2.1261e-01,  4.4577e-01],
          [-1.3351e+00,  6.2966e-01, -1.4650e+00,  ...,  2.9524e-02,
            6.4315e-01,  8.3389e-01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]],


        [[[-2.6698e-01, -8.8074e-02, -1.5340e-01,  ...,  3.3766e-03,
            2.8561e-01, -1.2721e-02],
          [-4.8673e-01, -2.5238

In [0]:
# 求加权和时每层的权重参数

In [41]:
s = nn.Parameter(torch.Tensor([1., 1., 1.]), requires_grad=True).cuda();s

tensor([1., 1., 1.], device='cuda:0', grad_fn=<CopyBackwards>)

In [0]:
# 权重和为1

In [43]:
s = F.softmax(s, dim=0);s

tensor([0.3333, 0.3333, 0.3333], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [0]:
# 求加权和时的相乘因子gamma

In [47]:
gamma = nn.Parameter(torch.Tensor(1, 1), requires_grad=True).cuda();gamma

tensor([[1.4041e-34]], device='cuda:0', grad_fn=<CopyBackwards>)

In [0]:
# 获得ELMo编码，维度为batch_size(2)*max_sentence_len(4)*1024

In [53]:
res = (all_layers[0]*s[0] + all_layers[1]*s[1] + all_layers[2]*s[2]) * gamma;res

tensor([[[-1.0259e-34, -7.6826e-35, -1.8838e-35,  ..., -1.0836e-35,
           4.3220e-35, -2.3216e-35],
         [-7.7731e-35, -4.3649e-35, -2.0179e-35,  ..., -7.1772e-36,
           1.9194e-34,  1.3312e-34],
         [-1.7425e-35, -9.3497e-35, -5.0731e-36,  ...,  4.8847e-35,
           4.5052e-35,  6.8358e-35],
         [-1.2432e-34, -4.8974e-35, -1.2754e-34,  ..., -1.3798e-35,
           4.7261e-36,  1.5037e-35]],

        [[-6.0207e-36, -1.8606e-34, -1.5361e-34,  ..., -8.2073e-35,
          -9.5748e-36, -6.6698e-36],
         [-9.0147e-35, -8.7691e-35, -1.1195e-34,  ...,  2.0585e-35,
           5.6011e-35,  3.1653e-35],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]], device='cuda:0', grad_fn=<MulBackward0>)

In [52]:
res.shape

torch.Size([2, 4, 1024])

#### 经过大规模语言模型预训练，ELMo获得了有效的上下文表示。与基于机器翻译的CoVe相比，ELMo能够充分利用预训练模型的参数。